In [ ]:
import argparse
import os

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.functions import col


In [4]:
parser = argparse.ArgumentParser()

In [5]:
# parser.add_argument('--input_table', required=True)
# parser.add_argument('--output')
# args = parser.parse_args()
# input_table = 'gs://ucl-premier-417919-bucket/par/part-00000-4b684bf2-1fdf-4fc9-a102-62b090bd6502-c000.snappy.parquet'
# output = args.output

In [6]:
credentials_location = "/home/bryan/.google/credentials/google_credentials.json"

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "../lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")


24/04/01 07:30:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
spark = SparkSession.builder \
    .appName('test') \
    .getOrCreate()

In [8]:
spark.conf.set('temporaryGcsBucket', 'dataproc-temp-us-west1-245998250410-jigg8mxm')

In [9]:
df_player_par = spark.read.parquet('gs://ucl-premier-417919-bucket/raw/player_stats.parquet')

In [10]:
df_player_par.show()

24/04/01 07:31:04 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------+------+------+---+--------------------+------------+---------+-------+------------+------------+----------+---------+------------+-------------+------+---------+--------+----------+---------+------------+-------+--------+-------+------------+-------+-------+-------+----------+---------+----------+-----+------+---------+-------+--------------+---------+-----------+----------+-----------+----------+
|              player|       country|height|weight|age|                club|ball_control|dribbling|marking|slide_tackle|stand_tackle|aggression|reactions|att_position|interceptions|vision|composure|crossing|short_pass|long_pass|acceleration|stamina|strength|balance|sprint_speed|agility|jumping|heading|shot_power|finishing|long_shots|curve|fk_acc|penalties|volleys|gk_positioning|gk_diving|gk_handling|gk_kicking|gk_reflexes|     value|
+--------------------+--------------+------+------+---+--------------------+------------+---------+-------+------------+----

In [11]:
df_player_par.printSchema()

root
 |-- player: binary (nullable = true)
 |-- country: string (nullable = true)
 |-- height: long (nullable = true)
 |-- weight: long (nullable = true)
 |-- age: long (nullable = true)
 |-- club: binary (nullable = true)
 |-- ball_control: long (nullable = true)
 |-- dribbling: long (nullable = true)
 |-- marking: string (nullable = true)
 |-- slide_tackle: long (nullable = true)
 |-- stand_tackle: long (nullable = true)
 |-- aggression: long (nullable = true)
 |-- reactions: long (nullable = true)
 |-- att_position: long (nullable = true)
 |-- interceptions: long (nullable = true)
 |-- vision: long (nullable = true)
 |-- composure: long (nullable = true)
 |-- crossing: long (nullable = true)
 |-- short_pass: long (nullable = true)
 |-- long_pass: long (nullable = true)
 |-- acceleration: long (nullable = true)
 |-- stamina: long (nullable = true)
 |-- strength: long (nullable = true)
 |-- balance: long (nullable = true)
 |-- sprint_speed: long (nullable = true)
 |-- agility: long (n

In [12]:
df_player_par = df_player_par.withColumn("player", col("player").cast("string")) \
       .withColumn("club", col("club").cast("string"))

In [13]:
df_player_par.printSchema()

root
 |-- player: string (nullable = true)
 |-- country: string (nullable = true)
 |-- height: long (nullable = true)
 |-- weight: long (nullable = true)
 |-- age: long (nullable = true)
 |-- club: string (nullable = true)
 |-- ball_control: long (nullable = true)
 |-- dribbling: long (nullable = true)
 |-- marking: string (nullable = true)
 |-- slide_tackle: long (nullable = true)
 |-- stand_tackle: long (nullable = true)
 |-- aggression: long (nullable = true)
 |-- reactions: long (nullable = true)
 |-- att_position: long (nullable = true)
 |-- interceptions: long (nullable = true)
 |-- vision: long (nullable = true)
 |-- composure: long (nullable = true)
 |-- crossing: long (nullable = true)
 |-- short_pass: long (nullable = true)
 |-- long_pass: long (nullable = true)
 |-- acceleration: long (nullable = true)
 |-- stamina: long (nullable = true)
 |-- strength: long (nullable = true)
 |-- balance: long (nullable = true)
 |-- sprint_speed: long (nullable = true)
 |-- agility: long (n

In [14]:
output_path = 'gs://ucl-premier-417919-bucket/par/'
df_player_par.coalesce(1).write.parquet(output_path,mode='overwrite')
spark.stop()
# df_green.write.parquet('gs://ucl-premier-417919-bucket/pq/')

In [15]:
# table_ref = 'premier-predict-417919.ucl_data.updated_types'
# df_player_table = spark.read.format('bigquery') \
#     .option("table", table_ref) \
#     .load()
# df_player_table.createOrReplaceTempView('df_player_table')